# gsim.meep — MEEP Photonic FDTD Simulation

This notebook demonstrates the `gsim.meep` module workflow:

1. Create a gdsfactory component
2. Configure simulation with fluent API (including **performance optimizations**)
3. Validate config
4. **Visualize 3D geometry** (client-side, no MEEP needed)
5. Write config (GDS + JSON + runner script) to disk
6. Inspect the generated files
7. **Server-side diagnostics** (geometry/field PNGs from MEEP runner)

> **Note:** MEEP runs on the cloud — no local MEEP installation needed.  
> Geometry is sent as a **GDS file** alongside a JSON config with layer stack info.

### Performance features

| Feature | Speedup | API |
|---------|---------|-----|
| DFT-convergence stopping | best for S-params | `sim.set_wavelength(..., stop_when_dft_decayed=True)` |
| Field-decay stopping | 1.5-3x | `sim.set_wavelength(..., stop_when_decayed=True)` |
| Polygon simplification | 10-100x fewer vertices | `sim.set_accuracy(simplify_tol=0.01)` |
| Subpixel averaging control | variable | `sim.set_accuracy(eps_averaging=False)` |
| Verbose progress stepping | observability | `sim.set_accuracy(verbose_interval=5.0)` |
| Eigenmode debug logging | diagnostics | Auto-saved `meep_debug.json` (n_eff, power conservation) |
| Server-side diagnostics | geometry validation | `sim.set_diagnostics(save_geometry=True, save_fields=True)` |
| Preview-only mode | seconds vs minutes | `sim.set_diagnostics(preview_only=True)` |
| Reduced default margins | 1.3-2x | Default `margin_xy/z` = 0.5 (was 1.0) |
| Fewer frequency points | ~2x | Default `num_freqs` = 11 (was 21) |
| `split_chunks_evenly=False` | MPI balance | Default in `SimConfig` |
| Port extension into PML | accuracy | Auto (default) via `sim.set_domain(extend_ports=...)` |

> **Note on symmetry:** `mp.Mirror` symmetries are **not used** for S-parameter extraction — MEEP's eigenmode coefficient normalization is incorrect when source monitors straddle a symmetry plane. This matches gplugins' approach. Symmetries are only applied in preview-only mode.

In [ ]:
import gdsfactory as gf
from ubcpdk import PDK, cells

PDK.activate()

## 1. Create a component

In [ ]:
component = cells.ebeam_y_1550()
component.plot()

In [ ]:
# Inspect ports
for p in component.ports:
    print(f"{p.name}: center={p.center}, width={p.width}, orientation={p.orientation}")

## 2. Configure MeepSim

In [ ]:
from gsim.meep import MeepSim

sim = MeepSim()

# Set the component geometry
sim.set_geometry(component)

# Set layer stack (from active PDK)
sim.set_stack()

# Configure domain margins and PML (defaults: margin=0.5, dpml=1.0)
# Waveguide ports are auto-extended into PML (extend_ports=0 → auto = margin_xy + dpml)
sim.set_domain(0.5)

# Crop z-domain tightly to photonic core region
sim.set_z_crop()

# Override material optical properties
sim.set_material("si", refractive_index=3.47)
sim.set_material("SiO2", refractive_index=1.44)

# Configure wavelength range
# DFT-convergence stopping: monitors all DFT monitors, has built-in time cap
sim.set_wavelength(
    wavelength=1.55,
    bandwidth=0.1,
    num_freqs=11,
    run_after_sources=200,
    stop_when_dft_decayed=True,
    decay_threshold=1e-3,
)

# Set MEEP grid resolution
# 30 pixels/um → ~13.4 pixels/wavelength in Si (n=3.47, λ_Si=0.45um)
# Minimum for accurate S-params; use 40+ for production
sim.set_resolution(pixels_per_um=30)

# Note: mp.Mirror symmetries are NOT used for S-parameter extraction
# (causes incorrect eigenmode normalization). This matches gplugins' approach.

# Accuracy and performance trade-offs
# - simplify_tol: reduce dense GDS polygon vertices (10nm tolerance)
# - verbose_interval: print progress every N MEEP time units
sim.set_accuracy(
    simplify_tol=0.01,
    eps_averaging=True,
    verbose_interval=5.0,
)

# Server-side diagnostics: geometry cross-section PNGs + field snapshot
# Set preview_only=True to skip FDTD and just validate geometry (seconds)
sim.set_diagnostics(save_geometry=True, save_fields=True)

# Output directory
sim.set_output_dir("./meep-sim-test")

## 3. Validate configuration

In [ ]:
result = sim.validate_config()
print(result)
print(f"Valid: {result.valid}")
if result.errors:
    print("Errors:", result.errors)
if result.warnings:
    print("Warnings:", result.warnings)

## 4. Inspect config models

In [ ]:
# FDTD config — wavelength/frequency conversion
print(f"Wavelength: {sim.fdtd_config.wavelength} um")
print(f"Bandwidth:  {sim.fdtd_config.bandwidth} um")
print(f"fcen:       {sim.fdtd_config.fcen:.4f} (1/um)")
print(f"df:         {sim.fdtd_config.df:.4f} (1/um)")
print(f"Num freqs:  {sim.fdtd_config.num_freqs}")

# Stopping config
stopping = sim.fdtd_config.stopping
print(f"\nStopping mode: {stopping.mode}")
print(f"  run_after_sources: {stopping.run_after_sources}")
if stopping.mode == "decay":
    print(f"  decay_by:     {stopping.decay_by}")
    print(f"  decay_dt:     {stopping.decay_dt}")
    print(f"  decay_component: {stopping.decay_component}")
elif stopping.mode == "dft_decay":
    print(f"  decay_by (tol):   {stopping.decay_by}")
    print(f"  dft_min_run_time: {stopping.dft_min_run_time}")
    print(f"  max_run_time:     {stopping.run_after_sources}")

# Symmetries
print(f"\nSymmetries: {len(sim.symmetries)}")
for s in sim.symmetries:
    print(f"  Mirror {s.direction}, phase={s.phase}")
print(f"split_chunks_evenly: {sim.split_chunks_evenly}")

# Domain config (including port extension)
dom = sim.domain_config
print(f"\nDomain config:")
print(f"  dpml:          {dom.dpml} um")
print(f"  margin_xy:     {dom.margin_xy} um")
print(f"  port_margin:   {dom.port_margin} um")
print(f"  extend_ports:  {dom.extend_ports} (0=auto: margin_xy + dpml = {dom.margin_xy + dom.dpml} um)")

# Accuracy config
acc = sim.accuracy_config
print(f"\nAccuracy config:")
print(f"  eps_averaging:     {acc.eps_averaging}")
print(f"  subpixel_maxeval:  {acc.subpixel_maxeval}")
print(f"  subpixel_tol:      {acc.subpixel_tol}")
print(f"  simplify_tol:      {acc.simplify_tol} um")
print(f"  verbose_interval:  {sim.verbose_interval} MEEP time units")

# Diagnostics config
diag = sim.diagnostics_config
print(f"\nDiagnostics config:")
print(f"  save_geometry:     {diag.save_geometry}")
print(f"  save_fields:       {diag.save_fields}")
print(f"  save_epsilon_raw:  {diag.save_epsilon_raw}")
print(f"  preview_only:      {diag.preview_only}")

In [ ]:
# Resolution config
print(f"Resolution: {sim.resolution_config.pixels_per_um} pixels/um")

# Resolution presets
from gsim.meep import ResolutionConfig

for name in ["coarse", "default", "fine"]:
    preset = getattr(ResolutionConfig, name)()
    print(f"  {name}: {preset.pixels_per_um} pixels/um")

## 5. Visualize geometry with simulation overlay

The `plot_2d()` and `plot_3d()` methods build a `GeometryModel` from the component + stack and render using solver-agnostic code.

When the stack and ports are configured, `plot_2d()` also draws:
- **Sim cell boundary** (dashed black) — geometry bbox + PML + margin
- **PML regions** — semi-transparent orange shading at cell edges
- **Source port** — red line with arrow showing excitation direction
- **Monitor ports** — blue lines at monitor locations

## 5. Visualize 3D geometry (client-side, no MEEP)

The `plot_2d()` and `plot_3d()` methods build a `GeometryModel` from the component + stack and render using solver-agnostic code.

In [ ]:
# 2D cross-section at z = center of core layer
sim.plot_2d(slices="z")

In [ ]:
# Multi-view: x, y, z cross-sections
sim.plot_2d(slices="xyz")

## 6. Write config to disk

`write_config()` generates three files:
- `layout.gds` — the GDS layout
- `sim_config.json` — layer stack, ports, materials, FDTD params
- `run_meep.py` — self-contained cloud runner script

In [ ]:
output_dir = sim.write_config()
print(f"Config written to: {output_dir}")

# List generated files
import os

for f in sorted(os.listdir(output_dir)):
    size = os.path.getsize(output_dir / f)
    print(f"  {f} ({size:,} bytes)")

## 9. Parse S-parameter results

In production, `sim.simulate()` sends the config to the cloud and returns parsed results.  
Here we demo the result parsing from CSV data.

The runner also saves `meep_debug.json` alongside `s_parameters.csv` with eigenmode diagnostics (n_eff, kdom, power conservation). This is auto-loaded into `result.debug_info` when parsing results.

In [ ]:
import numpy as np
from gsim.meep import SParameterResult

csv_data = output_dir / "s_parameters.csv"

# Parse results (auto-loads meep_debug.json if present)
result = SParameterResult.from_csv(csv_data)
print(f"Wavelengths: {len(result.wavelengths)} points")
print(f"S-params: {list(result.s_params.keys())}")
print(f"Port names: {result.port_names}")
print(f"Debug info loaded: {bool(result.debug_info)}")

# After a cloud run, debug_info contains eigenmode diagnostics:
if result.debug_info:
    meta = result.debug_info.get("metadata", {})
    print(f"\nSimulation metadata:")
    print(f"  Resolution: {meta.get('resolution')} pixels/um")
    print(f"  Cell size: {meta.get('cell_size')}")
    print(f"  Wall time: {meta.get('wall_seconds', 0):.1f}s")
    print(f"  Stopping mode: {meta.get('stopping_mode')}")

    # Check eigenmode n_eff — should be ~2.44 for Si waveguide at 1550nm
    for port, info in result.debug_info.get("eigenmode_info", {}).items():
        n_effs = info.get("n_eff", [])
        if n_effs:
            print(f"  Port {port} n_eff (center): {n_effs[len(n_effs)//2]:.3f}")

    # Power conservation — should be ~1.0
    pcons = result.debug_info.get("power_conservation", [])
    if pcons:
        print(f"  Power conservation (center): {pcons[len(pcons)//2]:.3f}")

In [ ]:
# Plot S-parameters (dB scale)
fig = result.plot(db=True)

## 9b. Server-side diagnostics

After a cloud run, the MEEP runner saves geometry cross-section PNGs and field snapshots alongside S-parameters. These are auto-detected and loaded into `result.diagnostic_images`.

| File | When | Content |
|------|------|---------|
| `meep_geometry_xy.png` | Pre-run (default) | XY cross-section: epsilon + sources + monitors + PML |
| `meep_geometry_xz.png` | Pre-run (3D only) | XZ cross-section: layer stack side view |
| `meep_geometry_yz.png` | Pre-run (3D only) | YZ cross-section: port face view |
| `meep_fields_xy.png` | Post-run (default) | Ey field overlaid on epsilon |

For **preview-only** mode (geometry validation without running FDTD), use `SParameterResult.from_directory()` since there's no CSV.

In [ ]:
# Display diagnostic images inline (after a cloud run)
# result.show_diagnostics()

# For full run: loads from CSV + auto-detects PNGs
# result = SParameterResult.from_csv("meep-sim-test/s_parameters.csv")
# result.show_diagnostics()

# For preview-only (no CSV): loads debug JSON + geometry PNGs
# result = SParameterResult.from_directory("meep-sim-test/")
# result.show_diagnostics()

# Check what diagnostic images are available
print(f"Diagnostic images: {list(result.diagnostic_images.keys())}")
for name, path in sorted(result.diagnostic_images.items()):
    print(f"  {name}: {path}")

## 10. Full workflow summary

```python
from gsim.meep import MeepSim

sim = MeepSim()
sim.set_geometry(component)
sim.set_stack()
sim.set_domain(0.5, dpml=1.0)        # 0.5um margins, 1um PML, auto-extends ports into PML
sim.set_z_crop()
sim.set_material("si", refractive_index=3.47)
sim.set_wavelength(
    wavelength=1.55, bandwidth=0.1,
    stop_when_dft_decayed=True,       # DFT convergence (best for S-params)
    run_after_sources=200,            # max time cap
)
sim.set_resolution(pixels_per_um=32)
sim.set_accuracy(
    simplify_tol=0.01,               # simplify dense GDS polygons (10nm)
    verbose_interval=5.0,             # progress prints every 5 MEEP time units
)
sim.set_diagnostics(save_geometry=True, save_fields=True)  # server-side PNGs
sim.set_output_dir("./meep-sim-test")

# Visualize before running (client-side: PML, ports, sim cell, dielectrics)
sim.plot_2d(slices="xyz")

# Run on GDSFactory+ cloud (requires auth)
result = sim.simulate()
result.plot()

# View server-side diagnostics (geometry + field PNGs from MEEP)
result.show_diagnostics()

# Check eigenmode diagnostics (auto-loaded from meep_debug.json)
if result.debug_info:
    for port, info in result.debug_info["eigenmode_info"].items():
        print(f"{port}: n_eff={info['n_eff'][len(info['n_eff'])//2]:.3f}")
    pcons = result.debug_info["power_conservation"]
    print(f"Power conservation: {pcons[len(pcons)//2]:.3f}")
```

### Port extension into PML

Waveguide ports are automatically extended through `margin_xy + dpml` into PML at `write_config()` time. This prevents spurious reflections from abrupt waveguide termination. To customize:

```python
sim.set_domain(0.5, extend_ports=2.0)  # explicit 2um extension
sim.set_domain(0.5, extend_ports=0.0)  # auto (default): margin_xy + dpml
```

### Preview-only mode (fast geometry validation)

```python
sim.set_diagnostics(preview_only=True)
sim.write_config()
result = sim.simulate()
result.show_diagnostics()  # geometry PNGs only — no fields, no S-params
```

### Stopping modes

| Mode | API | Best for |
|------|-----|----------|
| Fixed time | `run_after_sources=100` (default) | Known-duration sims |
| Field decay | `stop_when_decayed=True` | Point monitoring + time cap |
| DFT convergence | `stop_when_dft_decayed=True` | S-parameter extraction (recommended) |

### Note on symmetry

`mp.Mirror` symmetries are **not used** for S-parameter extraction. MEEP's `get_eigenmode_coefficients` with `add_mode_monitor` produces incorrect normalization when the source monitor straddles a symmetry plane (~2x coefficient error). This matches gplugins, which also never uses `mp.Mirror` for S-parameter runs. Symmetries are only applied in preview-only mode for fast geometry validation.

## 10b. Minimal (all defaults — 0.5um margins, 11 freqs, fixed stopping, auto port extension)

```python
from gsim.meep import MeepSim

sim = MeepSim()
sim.set_geometry(component)
sim.set_stack()
sim.set_z_crop()
sim.set_material("si", refractive_index=3.47)
sim.set_wavelength(wavelength=1.55, bandwidth=0.1)
sim.set_resolution(pixels_per_um=32)
sim.set_output_dir("./meep-sim-test")

# Optional: speed up dense GDS components
# sim.set_accuracy(simplify_tol=0.01, verbose_interval=5.0)

# Visualize before running
sim.plot_2d(slices="z")

# Run on GDSFactory+ cloud (requires auth)
result = sim.simulate()
result.plot()
```